In [1]:
### reading data and adding nodes as objects ###

branch_lengths_path="data/branchlength.dat"
tree_path="data/table.dat"
msa_path="data/msa.dat"


In [45]:
branch_lengths_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000013016_EHD3_NT.branchlength.dat"
tree_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000013016_EHD3_NT.table.dat"
msa_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000013016_EHD3_NT.msa.dat"

In [57]:
branch_lengths_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000112282_MED23_NT.branchlength.dat"
tree_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000112282_MED23_NT.table.dat"
msa_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000112282_MED23_NT.msa.dat"

In [68]:
branch_lengths_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000112984_KIF20A_NT.branchlength.dat"
tree_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000112984_KIF20A_NT.table.dat"
msa_path="/home/saisha/Desktop/UNIL/Semester_1/APP/Project/2023/mammals/ENSG00000112984_KIF20A_NT.msa.dat"


In [2]:

class TreeNode:
    def __init__(self, node_name):
        self.seq_leaf = None
        self.children = []
        self.parent = None
        self.branch_length = None
        self.lkl_list = None
        self.name = node_name

    def __len__(self):
        return len(self.seq_leaf)

    def getSequence(self):
        return self.seq_leaf

    def getBranchLength(self):
        return self.branch_length

    def getChildrenNodes(self):
        return self.children

    def get_lkl_list(self):
        return self.lkl_list

    def initialize_lkl_list(self):
        if self.seq_leaf:
            bases = {"A": [1, 0, 0, 0], "C": [0, 1, 0, 0], "G": [0, 0, 1, 0], "T": [0, 0, 0, 1]}
            self.lkl_list = [bases[base] for base in self.seq_leaf]
        else:
            self.lkl_list = None






def add_nodes_recursively(node, parent, node_dict, bl=None,sequence=None,lkl_list=None):
    if node not in node_dict:
        new_node = TreeNode(node)
        node_dict[node] = new_node

    if parent is not None:
        if parent not in node_dict:
            add_nodes_recursively(node=parent, parent=None, node_dict=node_dict)

        # Now, parent should be in node_dict
        node_dict[parent].children.append(node_dict[node])
        node_dict[node].parent = node_dict[parent]
        node_dict[node].branch_length=bl
        node_dict[node].seq_leaf=sequence
        node_dict[node].lkl_list=[]
        node_dict[node].initialize_lkl_list()
        

In [3]:
# Reading tree topology, branch lengths, and sequences
with open(tree_path, "r") as tree_file, open(branch_lengths_path, "r") as BL_file, open(msa_path, "r") as msa_file:
    line_num = 0
    

    bl = BL_file.readline().split(",")
    sequence = {}
    for line in msa_file:
        key, val = line.split()
        sequence[key] = val

    # Dictionary to store references to nodes by their names
    node_dict = {}

    for parent_child in tree_file:
        tmp_list = parent_child.split(",")

        parent_node_name = tmp_list[0]
        node_name = tmp_list[1].rstrip()

        branch_len = bl[line_num]

        if node_name in sequence.keys():
            leaf_seq = sequence[node_name]
        else:
            leaf_seq = None

        add_nodes_recursively(node=node_name, 
                              parent=parent_node_name, 
                              node_dict=node_dict,
                              bl=float(branch_len),
                              sequence=leaf_seq)


        line_num += 1


Tree_Data=[]
for key, value in node_dict.items():
    Tree_Data.append(value)

In [4]:
node_dict

{'1': <__main__.TreeNode at 0x7f4e54f15540>,
 '9': <__main__.TreeNode at 0x7f4e54f16980>,
 '2': <__main__.TreeNode at 0x7f4e54f14d30>}

In [7]:
from pprint import pprint
for key, value in node_dict.items():
    pprint(vars(value))
    break;

{'branch_length': 0.1,
 'children': [],
 'lkl_list': [[1, 0, 0, 0],
              [0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 1, 0, 0],
              [1, 0, 0, 0],
              [1, 0, 0, 0],
              [0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 1, 0, 0],
              [1, 0, 0, 0],
              [1, 0, 0, 0],
              [0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 1, 0, 0],
              [1, 0, 0, 0],
              [1, 0, 0, 0],
              [0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 1, 0, 0],
              [1, 0, 0, 0],
              [1, 0, 0, 0],
              [0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 1, 0, 0],
              [1, 0, 0, 0]],
 'name': '1',
 'parent': <__main__.TreeNode object at 0x7f4e54f16980>,
 'seq_leaf': 'AGATCAAGATCAAGATCAAGATCAAGATCA'}


In [18]:
## checking if data loaded correctly
for obj in node_dict:
  
    print("node type    : " , node_dict[obj].type)
    print("Parent name  : ",node_dict[obj].parent)
    print("current name : ",node_dict[obj].name)
    print("Child list   : ",node_dict[obj].children) ## same as node name
    print("Branch length: ",node_dict[obj].branch_length)
    print("sequence     : ", node_dict[obj].seq_leaf)
    print("lkl_list updated: " , node_dict[obj].get_lkl_list())



node type    :  external
Parent name  :  <__main__.TreeNode object at 0x7fea9c5c2f50>
current name :  1
Child list   :  []
Branch length:  0.1
sequence     :  AGATCAAGATCAAGATCAAGATCAAGATCA
lkl_list updated:  [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
node type    :  internal
Parent name  :  <__main__.TreeNode object at 0x7fea9c5c1ff0>
current name :  9
Child list   :  [<__main__.TreeNode object at 0x7fea7bf878e0>, <__main__.TreeNode object at 0x7fea9c5c3fd0>]
Branch length:  0.01
sequence     :  None
lkl_list updated:  None
node type    :  external
Parent name  :  <__main__.TreeNode object at 0x7fea9c5c2f50>
curre

In [19]:
## checking if data loaded correctly
for obj in Tree_Data:
  
    print("node type    : " , obj.type)
    print("Parent name  : ",obj.parent)
    print("current name : ",obj.name)
    print("Child list   : ",obj.children) ## same as node name
    print("Branch length: ",obj.branch_length)
    print("sequence     : ", obj.seq_leaf)
   
    print("lkl_list updated: " , obj.get_lkl_list())

node type    :  external
Parent name  :  <__main__.TreeNode object at 0x7fea9c5c2f50>
current name :  1
Child list   :  []
Branch length:  0.1
sequence     :  AGATCAAGATCAAGATCAAGATCAAGATCA
lkl_list updated:  [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
node type    :  internal
Parent name  :  <__main__.TreeNode object at 0x7fea9c5c1ff0>
current name :  9
Child list   :  [<__main__.TreeNode object at 0x7fea7bf878e0>, <__main__.TreeNode object at 0x7fea9c5c3fd0>]
Branch length:  0.01
sequence     :  None
lkl_list updated:  None
node type    :  external
Parent name  :  <__main__.TreeNode object at 0x7fea9c5c2f50>
curre

In [20]:
import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm
import math
mu=1 # USER INPUT
Q=np.array([[-3*mu, mu, mu, mu], [mu, -3*mu,  mu, mu], [ mu, mu, -3*mu, mu], [ mu, mu, mu, -3*mu]]) #symbolic of the model of evolution


## dict to store liklihoods of nucleotides at each site s in all nodes
lkl_dict = {}


## updating lkl lists for external nodes
for node in Tree_Data:
    print(node)

    if node.getSequence() is not None: # if it is not internal node 
        print(node.seq_leaf)
        print(node.get_lkl_list())
        lkl_dict[node.name]=node.get_lkl_list()



AGATCAAGATCAAGATCAAGATCAAGATCA
[[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
AGCTCAAGCTCAAGCTCAAGCTCAAGCTCA
[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
CGCTATCGCTATCGCTATCGCTATCGCTAT
[[0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1, 0, 0

In [21]:
lkl_dict

{'1': [[1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0]],
 '2': [[1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0]],
 '3': [[0, 1, 0, 0],
  [0, 0

In [22]:
### MAIN CALCULATION ###


def calculate_lkl_recursively(node,Q,lkl_dict):
    if node.seq_leaf is not None:
        return # skip leaf nodes
    
    childA=node.children[0]
    childB=node.children[1]
    parent_name=node.name
    childA_name=node.children[0].name
    childB_name=node.children[1].name
    if childA.name not in lkl_dict.keys():
        calculate_lkl_recursively(childA,Q,lkl_dict)
    if childB.name not in lkl_dict.keys():
        calculate_lkl_recursively(childB,Q,lkl_dict)
    
    anc_name=node.name
    anc_AB_lkl=calculate_lkl_anc(childA,childB,Q,lkl_dict)
    lkl_dict[anc_name]=anc_AB_lkl # updating lkl_dict
    #parent_nodes_visited.append(anc_name) # mark parent as vivited



def calculate_lkl_anc(childA,childB,Q,lkl_dict):
    childA_lkl = lkl_dict[childA.name] # we will get a list of lists with liklihood for each nuclleotide at each position
    childB_lkl = lkl_dict[childB.name]
    anc_AB_lkl=[]
    anc_name=childA.parent.name #could likewise be childB.parent
    # calculating the liklihood of all 4 nucleotides at each position for the parent
    for pos_list in range(len(childA_lkl)): #using all positions in both children nodes, calculate ancetral nucleotide liklihoods
        P1 = scipy.linalg.expm(Q * float(childA.getBranchLength()))  
        P2 = scipy.linalg.expm(Q * float(childB.getBranchLength()))  
        vec1=np.matmul(childA_lkl[pos_list],P1)
        vec2=np.matmul(childB_lkl[pos_list],P2)

        vec_anc = [vec1[i] * vec2[i] for i in range(4)]
        ## !! CHECKPOINT !! ##
        #print("node",childA.parent.name,vec_anc) ## we are geiting 4 values/pos i.e lkl of all 4 nucleotides at each position.

        anc_AB_lkl.append(np.array(vec_anc)) ## position lkl to main list for this node
    return(anc_AB_lkl)


#parent_nodes_visited=[]
for root_node in Tree_Data:
    calculate_lkl_recursively(root_node, Q, lkl_dict)
    print("root_node",root_node.name)



'''
for childA in Tree_Data:
    for childB in Tree_Data:
        if childA != childB: ## if not the same node
            # get two siblings from the tree i.e nodes having same parent
            if childA.parent == childB.parent and not (childA.parent.name in parent_nodes_visited):
                print(childA.parent.name,":",childA.name,"-",childB.name)
                #parent_nodes_visited.append(childA.parent.name) # mark parent as vivited

                #break;
                anc_name=childA.parent.name
                anc_AB_lkl=calculate_lkl_anc(childA,childB,lkl_dict)
                lkl_dict[anc_name]=anc_AB_lkl # updating lkl_dict
                parent_nodes_visited.append(anc_name) # mark parent as vivited


'''

root_node 1
root_node 9
root_node 2
root_node 8
root_node 3
root_node 4
root_node 7
root_node 5
root_node 6


'\nfor childA in Tree_Data:\n    for childB in Tree_Data:\n        if childA != childB: ## if not the same node\n            # get two siblings from the tree i.e nodes having same parent\n            if childA.parent == childB.parent and not (childA.parent.name in parent_nodes_visited):\n                print(childA.parent.name,":",childA.name,"-",childB.name)\n                #parent_nodes_visited.append(childA.parent.name) # mark parent as vivited\n\n                #break;\n                anc_name=childA.parent.name\n                anc_AB_lkl=calculate_lkl_anc(childA,childB,lkl_dict)\n                lkl_dict[anc_name]=anc_AB_lkl # updating lkl_dict\n                parent_nodes_visited.append(anc_name) # mark parent as vivited\n\n\n'

In [23]:
lkl_dict

{'1': [[1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0]],
 '2': [[1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0]],
 '3': [[0, 1, 0, 0],
  [0, 0

In [24]:
for node in lkl_dict:
    npmm_sum=0
    for pos_list in lkl_dict[node]:
        npmm=np.matmul(pos_list,[0.25,0.25,0.25,0.25])
        npmm_sum+=math.log(npmm)
    print("Node ",node,"npmm: ",npmm_sum)

### Liklihood of Node 6 is the liklihod of observing the tree

Node  1 npmm:  -41.588830833596695
Node  2 npmm:  -41.588830833596695
Node  3 npmm:  -41.588830833596695
Node  4 npmm:  -41.588830833596695
Node  5 npmm:  -41.588830833596695
Node  9 npmm:  -75.38590510870945
Node  8 npmm:  -124.33743778967978
Node  7 npmm:  -68.08960098928449
Node  6 npmm:  -182.05582421991764
